# Контекст

- NER - классификация именнованных токенов
- bert модель
- conll2003 - dataset 

In [72]:
import numpy as np
import pandas as pd
import datasets
from transformers import BertTokenizerFast,\
                         AutoModelForTokenClassification,\
                         TrainingArguments,\
                         Trainer,\
                         DataCollatorForTokenClassification,\
                         pipeline
from sklearn.metrics import precision_recall_fscore_support,\
                            accuracy_score

# Чтение файла

In [50]:
conll2003 = datasets.load_dataset('conll2003')

In [51]:
conll2003['train'][0:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

In [52]:
conll2003['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

# Токенизация и выравнивание

In [53]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [54]:
def tokenization_align_labels(
        
        examples,  # dict {'tokens': ...., 'ner_tags: .... и др'}
        label_all_tokens=False

        ):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) # [None 1 2 3 4 5 6 None]
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Пример использования функции
print(conll2003['train'][0:2]['tokens'])
tokenization_align_labels(conll2003['train'][0:2], True)['labels'] # на выходе нормализованные индексы токенов

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn']]


[[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100]]

In [65]:
tokenized_datasets = conll2003.map(tokenization_align_labels, batched = True)

# Скачивание модели

In [56]:
model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 9 # кол-во имен собственных в данных
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training args

In [57]:
training_args = TrainingArguments(

        output_dir='training/model_points',
        do_train=True,
        do_eval=True,
        num_train_epochs=3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        warmup_steps=100,
        weight_decay=0.01,
        logging_strategy='steps',
        logging_dir='training/logs',
        logging_steps=50,
        evaluation_strategy="steps",
        save_steps = 500,
        fp16=True,
        load_best_model_at_end=True,
        report_to = 'wandb',
        learning_rate = 0.00002

)

In [58]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Метрики

In [69]:
metric = datasets.load_metric('seqeval')
label_list = conll2003["train"].features["ner_tags"].feature.names 
label_list

c:\Users\oudel\anaconda3\envs\newenv\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [70]:
def compute_metrics(p):
    predictions, labels = p 
    predictions = np.argmax(predictions, axis=2) 

    true_predictions = [ 
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(predictions, labels) 
                        ] 
    
    true_labels = [ 
      [label_list[l] for (p, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(predictions, labels) 
                  ] 
    
    results = metric.compute(predictions=true_predictions, references=true_labels)


    return { 
                "precision": results["overall_precision"], 
                "recall": results["overall_recall"], 
                "f1": results["overall_f1"], 
                "accuracy": results["overall_accuracy"], 
           } 

# Trainer

In [71]:
trainer = Trainer(

        model = model,
        args = training_args,
        train_dataset = tokenized_datasets['train'],
        eval_dataset = tokenized_datasets['validation'],
        compute_metrics = compute_metrics,
        tokenizer = tokenizer,
        data_collator = data_collator

 )

trainer.train()

c:\Users\oudel\anaconda3\envs\newenv\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/1317 [00:00<?, ?it/s]

{'loss': 0.1273, 'grad_norm': 1.7592236995697021, 'learning_rate': 1e-05, 'epoch': 0.11}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.11439711600542068, 'eval_precision': 0.8167369445345443, 'eval_recall': 0.8475260854930999, 'eval_f1': 0.8318467129170796, 'eval_accuracy': 0.9708734083563724, 'eval_runtime': 2.9826, 'eval_samples_per_second': 1089.636, 'eval_steps_per_second': 17.099, 'epoch': 0.11}
{'loss': 0.083, 'grad_norm': 1.4185106754302979, 'learning_rate': 2e-05, 'epoch': 0.23}


...

  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.04520042613148689, 'eval_precision': 0.9315362989912354, 'eval_recall': 0.947997307303938, 'eval_f1': 0.9396947201601468, 'eval_accuracy': 0.9882403333203535, 'eval_runtime': 3.0305, 'eval_samples_per_second': 1072.443, 'eval_steps_per_second': 16.829, 'epoch': 2.85}
{'loss': 0.0208, 'grad_norm': 0.2904956042766571, 'learning_rate': 2.7937551355792936e-07, 'epoch': 2.96}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.04488206282258034, 'eval_precision': 0.9347717842323652, 'eval_recall': 0.9478290138000673, 'eval_f1': 0.9412551182418317, 'eval_accuracy': 0.9885518476694832, 'eval_runtime': 3.0067, 'eval_samples_per_second': 1080.926, 'eval_steps_per_second': 16.962, 'epoch': 2.96}
{'train_runtime': 183.21, 'train_samples_per_second': 229.917, 'train_steps_per_second': 7.188, 'train_loss': 0.04753323887095093, 'epoch': 3.0}


TrainOutput(global_step=1317, training_loss=0.04753323887095093, metrics={'train_runtime': 183.21, 'train_samples_per_second': 229.917, 'train_steps_per_second': 7.188, 'train_loss': 0.04753323887095093, 'epoch': 3.0})

# Проверка

In [74]:
q=[trainer.evaluate(eval_dataset = tokenized_datasets[data]) for data in ['train', 'validation', 'test']]
pd.DataFrame(q, index=["train","val", 'test']).iloc[:,:5]

  0%|          | 0/220 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy
train,0.016475,0.972156,0.977659,0.974900,0.995747
val,0.046769,0.930824,0.948839,0.939745,0.988007
test,0.101807,0.885808,0.910588,0.898027,0.978723


In [75]:
text = 'Moscow never sleeps. But Dima sleeps'

from transformers import pipeline
nlp = pipeline("ner", model = model, tokenizer=tokenizer)

ner_results = nlp(text)
print(ner_results)

[{'entity': 'LABEL_5', 'score': 0.99672866, 'index': 1, 'word': 'moscow', 'start': 0, 'end': 6}, {'entity': 'LABEL_0', 'score': 0.99969566, 'index': 2, 'word': 'never', 'start': 7, 'end': 12}, {'entity': 'LABEL_0', 'score': 0.9996524, 'index': 3, 'word': 'sleeps', 'start': 13, 'end': 19}, {'entity': 'LABEL_0', 'score': 0.99969065, 'index': 4, 'word': '.', 'start': 19, 'end': 20}, {'entity': 'LABEL_0', 'score': 0.9996772, 'index': 5, 'word': 'but', 'start': 21, 'end': 24}, {'entity': 'LABEL_1', 'score': 0.997707, 'index': 6, 'word': 'dim', 'start': 25, 'end': 28}, {'entity': 'LABEL_1', 'score': 0.9791077, 'index': 7, 'word': '##a', 'start': 28, 'end': 29}, {'entity': 'LABEL_0', 'score': 0.99958986, 'index': 8, 'word': 'sleeps', 'start': 30, 'end': 36}]


In [89]:
print(ner_results[0])
print(ner_results[5])
print(label_list)

{'entity': 'LABEL_5', 'score': 0.99672866, 'index': 1, 'word': 'moscow', 'start': 0, 'end': 6}
{'entity': 'LABEL_1', 'score': 0.997707, 'index': 6, 'word': 'dim', 'start': 25, 'end': 28}
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
